# 한국/미국 금리 조회 Tool 테스트

이 노트북은 FRED API(미국)와 ECOS API(한국)를 사용하여 금리 데이터를 조회하는 LangChain tool을 테스트합니다.

In [9]:
from dotenv import load_dotenv
import os

load_dotenv() # 모든 환경 변수를 한 번에 로드

# FRED API 키 사용
FRED_API_KEY = os.getenv("FRED_API_KEY")

# ECOS API 키 사용
ECOS_API_KEY = os.getenv("ECOS_API_KEY") 

print(f"FRED 키 로드 완료: {FRED_API_KEY is not None}")
print(f"ECOS 키 로드 완료: {ECOS_API_KEY is not None}")

FRED 키 로드 완료: True
ECOS 키 로드 완료: True


## 1. 미국 기준금리 조회

In [10]:
from tools.interest_rate_tool import get_us_interest_rate

# 최근 12개월 미국 기준금리
result = get_us_interest_rate.invoke({
    "rate_type": "base_rate",
    "months": 12
})
print(result)

FRED API 요청 중 오류: 400 Client Error: Bad Request for url: https://api.stlouisfed.org/fred/series/observations?series_id=FEDFUNDS&api_key=07b7c58f18be38e51c0e63b51b9a6995+%23+%EB%AF%B8%EA%B5%AD+%EC%97%B0%EB%B0%A9+%EA%B8%88%EB%A6%AC&file_type=json&frequency=m&observation_start=2024-11-03
미국 기준금리: 데이터 없음


## 2. 미국 10년물 국채 수익률 조회

In [11]:
# 최근 12개월 미국 10년물 국채 수익률
result = get_us_interest_rate.invoke({
    "rate_type": "10y_treasury",
    "months": 12
})
print(result)

FRED API 요청 중 오류: 400 Client Error: Bad Request for url: https://api.stlouisfed.org/fred/series/observations?series_id=DGS10&api_key=07b7c58f18be38e51c0e63b51b9a6995+%23+%EB%AF%B8%EA%B5%AD+%EC%97%B0%EB%B0%A9+%EA%B8%88%EB%A6%AC&file_type=json&frequency=m&observation_start=2024-11-03
미국 10년물 국채 수익률: 데이터 없음


## 3. 한국 기준금리 조회

In [4]:
from tools.interest_rate_tool import get_korea_interest_rate

# 최근 12개월 한국 기준금리
result = get_korea_interest_rate.invoke({"months": 12})
print(result)

ECOS API 요청 중 오류: 404 Client Error: Not Found for url: https://ecos.bok.or.kr/api/StatisticSearch/5R7Q6RTQNXYZJO92A9F1%20#%20%ED%95%9C%EA%B5%AD%20%EA%B8%88%EB%A6%AC/json/kr/1/1000/722Y001/M/202411/202510
한국 기준금리: 데이터 없음


## 4. 한미 금리 비교

In [5]:
from tools.interest_rate_tool import compare_korea_us_rates

# 최근 12개월 한미 금리 비교
result = compare_korea_us_rates.invoke({"months": 12})
print(result)

FRED API 요청 중 오류: 400 Client Error: Bad Request for url: https://api.stlouisfed.org/fred/series/observations?series_id=FEDFUNDS&api_key=07b7c58f18be38e51c0e63b51b9a6995+%23+%EB%AF%B8%EA%B5%AD+%EC%97%B0%EB%B0%A9+%EA%B8%88%EB%A6%AC&file_type=json&frequency=m&observation_start=2024-11-03
ECOS API 요청 중 오류: 404 Client Error: Not Found for url: https://ecos.bok.or.kr/api/StatisticSearch/5R7Q6RTQNXYZJO92A9F1%20#%20%ED%95%9C%EA%B5%AD%20%EA%B8%88%EB%A6%AC/json/kr/1/1000/722Y001/M/202411/202510
오류: 데이터 조회 실패


## 5. InterestRateTool 클래스 직접 사용

In [6]:
from tools.interest_rate_tool import InterestRateTool

# 도구 초기화
rate_tool = InterestRateTool()

# 미국 기준금리 데이터 가져오기
us_data = rate_tool.get_us_rates("base_rate", months=6)
print(f"미국 기준금리 데이터: {len(us_data)}개")
print(us_data[-3:])  # 최근 3개월만 출력

FRED API 요청 중 오류: 400 Client Error: Bad Request for url: https://api.stlouisfed.org/fred/series/observations?series_id=FEDFUNDS&api_key=07b7c58f18be38e51c0e63b51b9a6995+%23+%EB%AF%B8%EA%B5%AD+%EC%97%B0%EB%B0%A9+%EA%B8%88%EB%A6%AC&file_type=json&frequency=m&observation_start=2025-05-02
미국 기준금리 데이터: 0개
[]


In [7]:
# 한국 기준금리 데이터 가져오기
kr_data = rate_tool.get_korea_rate(months=6)
print(f"한국 기준금리 데이터: {len(kr_data)}개")
print(kr_data[-3:])  # 최근 3개월만 출력

ECOS API 요청 중 오류: 404 Client Error: Not Found for url: https://ecos.bok.or.kr/api/StatisticSearch/5R7Q6RTQNXYZJO92A9F1%20#%20%ED%95%9C%EA%B5%AD%20%EA%B8%88%EB%A6%AC/json/kr/1/1000/722Y001/M/202505/202510
한국 기준금리 데이터: 0개
[]


## 6. 데이터 시각화

In [8]:
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import font_manager, rc

# 한글 폰트 설정 (Windows)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 데이터 조회
rate_tool = InterestRateTool()
us_data = rate_tool.get_us_rates("base_rate", months=24)
kr_data = rate_tool.get_korea_rate(months=24)

# DataFrame 생성
us_df = pd.DataFrame(us_data)
kr_df = pd.DataFrame(kr_data)

# 날짜 형식 변환
us_df['date'] = pd.to_datetime(us_df['date'])
kr_df['date'] = pd.to_datetime(kr_df['date'], format='%Y%m')

# 시각화
plt.figure(figsize=(14, 7))

plt.plot(us_df['date'], us_df['value'], 
         label='미국 기준금리', color='red', linewidth=2, marker='o')
plt.plot(kr_df['date'], kr_df['value'], 
         label='한국 기준금리', color='blue', linewidth=2, marker='s')

# 금리 역전 구간 강조
# (여기서는 간단하게 처리)

plt.title('한미 기준금리 추이 비교 (최근 24개월)', fontsize=16)
plt.xlabel('날짜', fontsize=12)
plt.ylabel('금리 (%)', fontsize=12)
plt.legend(fontsize=10, loc='upper left')
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

FRED API 요청 중 오류: 400 Client Error: Bad Request for url: https://api.stlouisfed.org/fred/series/observations?series_id=FEDFUNDS&api_key=07b7c58f18be38e51c0e63b51b9a6995+%23+%EB%AF%B8%EA%B5%AD+%EC%97%B0%EB%B0%A9+%EA%B8%88%EB%A6%AC&file_type=json&frequency=m&observation_start=2023-11-09
ECOS API 요청 중 오류: 404 Client Error: Not Found for url: https://ecos.bok.or.kr/api/StatisticSearch/5R7Q6RTQNXYZJO92A9F1%20#%20%ED%95%9C%EA%B5%AD%20%EA%B8%88%EB%A6%AC/json/kr/1/1000/722Y001/M/202311/202510


KeyError: 'date'

## 7. LangChain Agent와 통합

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate
from tools.interest_rate_tool import get_interest_rate_tools

# LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 도구 목록
tools = get_interest_rate_tools()

# 프롬프트 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 글로벌 금리 분석 전문가입니다.
한국과 미국의 금리 데이터를 분석하여 경제 동향을 파악합니다.
제공된 도구를 활용하여 사용자의 질문에 답변하세요."""),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# Agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# 테스트 질문
response = agent_executor.invoke({
    "input": "현재 한국과 미국의 기준금리를 비교하고, 금리 역전 상황을 설명해주세요."
})

print("\n" + "="*80)
print("Agent 응답:")
print("="*80)
print(response["output"])

## 8. 다양한 기간으로 테스트

In [ ]:
# 6개월, 12개월, 24개월 비교
periods = [6, 12, 24]

for period in periods:
    print(f"\n{'='*80}")
    print(f"최근 {period}개월 한미 금리 비교")
    print('='*80)
    
    result = compare_korea_us_rates.invoke({"months": period})
    print(result)

## 9. 금리 추세 분석

In [ ]:
# 금리 추세 계산 함수
def analyze_trend(data, name):
    if len(data) < 2:
        print(f"{name}: 데이터 부족")
        return
    
    first = data[0]['value']
    last = data[-1]['value']
    change = last - first
    change_pct = (change / first * 100) if first != 0 else 0
    
    print(f"\n{name}:")
    print(f"  시작: {first}% ({data[0]['date']})")
    print(f"  종료: {last}% ({data[-1]['date']})")
    print(f"  변화: {change:+.2f}%p ({change_pct:+.2f}%)")
    print(f"  추세: {'상승' if change > 0 else '하락' if change < 0 else '보합'}")

# 24개월 데이터 분석
rate_tool = InterestRateTool()
us_data = rate_tool.get_us_rates("base_rate", months=24)
kr_data = rate_tool.get_korea_rate(months=24)
treasury_data = rate_tool.get_us_rates("10y_treasury", months=24)

print("="*80)
print("금리 추세 분석 (최근 24개월)")
print("="*80)

analyze_trend(us_data, "미국 기준금리")
analyze_trend(kr_data, "한국 기준금리")
analyze_trend(treasury_data, "미국 10년물 국채")